In [6]:
import pandas as pd

In [8]:
raw_data = pd.read_csv('/workspaces/Wikistim-Summarization/completed_papers_wikistim.csv')

In [9]:
raw_data.head(1)

,PUBLICATION INFORMATION,Author(s),Title,Journal,"Volume, issue, pages",Year,Full text link,PUBMED link,STUDY DESCRIPTION AND METHODS,Study design,...,Physician experience,Facility location,Facility type,DATA EXTRACTION,Awaiting completion. You may use the above link to download a CSV,Checked out by/date due,Submitted by/date submitted,Submission reviewed by/date reviewed,Unnamed: 214,Unnamed: 215
0,,"Al-Kaisy A, Van Buyten JP, Carganillo R, Caraw...","10 kHz SCS therapy for chronic pain, effects o...",Sci Rep,9(1):11441,2019,https://www.nature.com/articles/s41598-019-477...,https://www.ncbi.nlm.nih.gov/pubmed/31391503,,Post hoc analysis of data from two published s...,...,NaN,NaN,NaN,,NaN,NaN,Anand Rotte/3 Sept 2019,JBS/finalized May 2020,NaN,NaN


In [27]:
data_with_url = raw_data[~raw_data['Full text link'].isnull()]

In [28]:
import requests
import os
import time

# Create a directory to store the downloaded PDFs if it doesn't exist
download_dir = "pdf_downloads3"
os.makedirs(download_dir, exist_ok=True)

# User-agent to mimic a web browser request
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"

# Maximum number of retries
max_retries = 3

# Create a new column in data_with_url to store the file paths
data_with_url['Downloaded PDF Path'] = ""

# Iterate through the URLs and download the PDFs
for idx, url in data_with_url['Full text link'].items():
    retry_count = 0
    while retry_count < max_retries:
        try:
            headers = {"User-Agent": user_agent}
            response = requests.get(url, headers=headers, timeout=30)
            
            # Check if the response status code indicates a successful download
            if response.status_code == 200:
                # Define the filename based on the index
                filename = f"pdf_{idx}.pdf"
                # Define the full path to save the file
                file_path = os.path.join(download_dir, filename)
                
                # Save the PDF to the specified directory with the indexed filename
                with open(file_path, 'wb') as pdf_file:
                    pdf_file.write(response.content)
                
                # Update the DataFrame with the downloaded PDF path
                data_with_url.at[idx, 'Downloaded PDF Path'] = file_path
                
                print(f"Downloaded PDF {idx} to {file_path}")
                break  # Successful download, exit retry loop
            else:
                print(f"Failed to download PDF {idx} from URL {url}. Status Code: {response.status_code}")
        except Exception as e:
            print(f"Error downloading PDF {idx} from URL {url}: {e}")
        
        # Increment the retry count and wait before the next retry
        retry_count += 1
        time.sleep(5)  # Wait for 5 seconds before retrying

    if retry_count >= max_retries:
        print(f"Maximum retry count reached for PDF {idx}. Download failed.")


/tmp/ipykernel_5918/2989735423.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_with_url['Downloaded PDF Path'] = ""


Downloaded PDF 0 to pdf_downloads3/pdf_0.pdf
Downloaded PDF 1 to pdf_downloads3/pdf_1.pdf
Downloaded PDF 3 to pdf_downloads3/pdf_3.pdf
Failed to download PDF 4 from URL http://onlinelibrary.wiley.com/doi/10.1111/ner.12746/epdf. Status Code: 403
Failed to download PDF 4 from URL http://onlinelibrary.wiley.com/doi/10.1111/ner.12746/epdf. Status Code: 403
Failed to download PDF 4 from URL http://onlinelibrary.wiley.com/doi/10.1111/ner.12746/epdf. Status Code: 403
Maximum retry count reached for PDF 4. Download failed.
Downloaded PDF 5 to pdf_downloads3/pdf_5.pdf
Downloaded PDF 6 to pdf_downloads3/pdf_6.pdf
Failed to download PDF 7 from URL https://www.jpsmjournal.com/article/S0885-3924(96)00322-3/pdf. Status Code: 403
Failed to download PDF 7 from URL https://www.jpsmjournal.com/article/S0885-3924(96)00322-3/pdf. Status Code: 403
Failed to download PDF 7 from URL https://www.jpsmjournal.com/article/S0885-3924(96)00322-3/pdf. Status Code: 403
Maximum retry count reached for PDF 7. Download

In [29]:
import fitz  # PyMuPDF

# Create a new column in data_with_url to store the extracted text
data_with_url['full_article_text'] = ""

# Iterate through the rows in data_with_url
for idx, row in data_with_url.iterrows():
    pdf_filename = row['Downloaded PDF Path']
    if pdf_filename and pdf_filename.endswith(".pdf"):  # Check if PDF filename is not empty and ends with .pdf
        pdf_path = f"/workspaces/Wikistim-Summarization/{pdf_filename}"  # Add the prefix to the PDF path
        try:
            doc = fitz.open(pdf_path)
            # Extract text from all pages and join them into a single string
            full_text = ""
            for page in doc:
                full_text += page.get_text()
            
            # Check if the extracted text is truly empty (no visible characters)
            if full_text.strip():
                # Update the DataFrame with the extracted text
                data_with_url.at[idx, 'full_article_text'] = full_text
            else:
                print(f"Skipped processing {pdf_path} because extracted text is empty.")
        except Exception as e:
            print(f"Skipped processing {pdf_path} due to an error: {e}")
            continue  # Skip this document and move to the next one

# Filter out rows where 'full_article_text' is truly empty
filtered_data = data_with_url[data_with_url['full_article_text'].str.strip() != ""]



Skipped processing /workspaces/Wikistim-Summarization/pdf_downloads3/pdf_3.pdf due to an error: cannot open broken document


/tmp/ipykernel_5918/3164694365.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_with_url['full_article_text'] = ""


Skipped processing /workspaces/Wikistim-Summarization/pdf_downloads3/pdf_17.pdf due to an error: cannot open broken document
Skipped processing /workspaces/Wikistim-Summarization/pdf_downloads3/pdf_23.pdf due to an error: cannot open broken document
Skipped processing /workspaces/Wikistim-Summarization/pdf_downloads3/pdf_27.pdf due to an error: cannot open broken document
Skipped processing /workspaces/Wikistim-Summarization/pdf_downloads3/pdf_29.pdf due to an error: cannot open broken document
Skipped processing /workspaces/Wikistim-Summarization/pdf_downloads3/pdf_46.pdf due to an error: cannot open broken document
Skipped processing /workspaces/Wikistim-Summarization/pdf_downloads3/pdf_51.pdf due to an error: cannot open broken document
Skipped processing /workspaces/Wikistim-Summarization/pdf_downloads3/pdf_62.pdf due to an error: cannot open broken document
Skipped processing /workspaces/Wikistim-Summarization/pdf_downloads3/pdf_68.pdf due to an error: cannot open broken document


In [30]:
data_with_full_text = filtered_data[~filtered_data['full_article_text'].isnull()]

In [31]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string

# Download NLTK data (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Define a function for text preprocessing
def preprocess_text(text):
    # Lowercasing
    text = text.lower()

    # Tokenization (splitting text into words)
    tokens = word_tokenize(text)

    # Removing stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    punctuation = set(string.punctuation)

    filtered_tokens = [word for word in tokens if word not in stop_words and word not in punctuation]

    # Lemmatization (using WordNet Lemmatizer)
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

    # Joining tokens back into a string
    preprocessed_text = ' '.join(lemmatized_tokens)

    return preprocessed_text

# Apply the preprocessing function to the 'full_article_text' column
data_with_full_text['full_article_text_cleaned'] = data_with_full_text['full_article_text'].apply(preprocess_text)


[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [34]:
data_with_full_text['full_article_text_cleaned'][1]

'research article multicenter real-world review 10 khz sc outcome treatment chronic trunk and/or limb pain thomas stauss1 faycal el majdoub2 dawood sayed3 gernot surges4 william s. rosenberg5 leonardo kapural6 richard bundschu7 abdul lalkhen8 nileshkumar patel1 bradford gliner9 jeyakumar subbaroyan9 anand rotte9 deborah r. edgar10 martin bettag4 mohammad maarouf2 1advanced pain management greenﬁeld wisconsin 2department stereotactic functional neurosurgery cologne merheim medical center university witten/herdecke cologne germany 3department anesthesiology pain medicine university kansa medical center kansa city kansa 4kh barmherzige br€uder trier germany 5center relief pain kansa city missouri 6carolinas pain institute winston-salem north carolina 7coastal orthopedics pain medicine bradenton florida 8the manchester salford pain centre salford united kingdom 9nevro corp redwood city california 10commexus ltd dunblane united kingdom correspondence anand rotte nevro corp redwood city ca t

In [35]:
data_with_full_text

,PUBLICATION INFORMATION,Author(s),Title,Journal,"Volume, issue, pages",Year,Full text link,PUBMED link,STUDY DESCRIPTION AND METHODS,Study design,...,DATA EXTRACTION,Awaiting completion. You may use the above link to download a CSV,Checked out by/date due,Submitted by/date submitted,Submission reviewed by/date reviewed,Unnamed: 214,Unnamed: 215,Downloaded PDF Path,full_article_text,full_article_text_cleaned
0,,"Al-Kaisy A, Van Buyten JP, Carganillo R, Caraw...","10 kHz SCS therapy for chronic pain, effects o...",Sci Rep,9(1):11441,2019,https://www.nature.com/articles/s41598-019-477...,https://www.ncbi.nlm.nih.gov/pubmed/31391503,,Post hoc analysis of data from two published s...,...,,NaN,NaN,Anand Rotte/3 Sept 2019,JBS/finalized May 2020,NaN,NaN,pdf_downloads3/pdf_0.pdf,1\nScientific RepoRtS | (2019) 9:11441...,1 scientific report 2019 9:11441 http //doi.or...
1,,"Stauss T, El Majdoub F, Sayed D, Surges G, Ros...",A multicenter real-world review of 10 kHz SCS ...,Ann Clin Transl Neurol,6(3):496-507,2019,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,https://www.ncbi.nlm.nih.gov/pubmed/30911573,,"Multicenter, retrospective analysis with no co...",...,,NaN,NaN,"Anand Rotte, M Pharm, PhD",Jane Shipley July 2019,NaN,NaN,pdf_downloads3/pdf_1.pdf,RESEARCH ARTICLE\nA multicenter real-world rev...,research article multicenter real-world review...
5,,"Sinclair NC, McDermott HJ, Bullus KJ, Fallon J...",Subthalamic nucleus deep brain stimulation evo...,Ann Neurol,83(5):1027-1031,2018,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,https://www.ncbi.nlm.nih.gov/pubmed/29727475,,Prospective clinical study,...,,NaN,NaN,Bionics Institute/June 2018,JBS/June 2018,NaN,NaN,pdf_downloads3/pdf_5.pdf,Subthalamic Nucleus Deep Brain Stimulation Evo...,subthalamic nucleus deep brain stimulation evo...
6,,"Amirdelfan K, Yu C, Doust MW, Gliner BE, Morga...",Long-term quality of life improvement for chro...,Qual Life Res,27(8):2035-2044,2018,https://link.springer.com/content/pdf/10.1007/...,https://www.ncbi.nlm.nih.gov/pubmed/29858746,,Multicenter 1:1 RCT,...,,NaN,NaN,Anand Rotte on behalf of Nevro/ 12 July 2018,JBS/July 2018,NaN,NaN,pdf_downloads3/pdf_6.pdf,Vol.:(0123456789)\n1 3\nQuality of Life Resear...,vol 0123456789 1 3 quality life research 2018 ...
11,,Simpson BA.,Spinal cord stimulation in 60 cases of intract...,J Neurol Neurosurg Psychiatry,54(3):196-199,1991,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,http://www.ncbi.nlm.nih.gov/pubmed/1827648,,Retrospective,...,,NaN,NaN,JBS,NaN,NaN,NaN,pdf_downloads3/pdf_11.pdf,"Journal ofNeurology, Neurosurgery, and Psychia...",journal ofneurology neurosurgery psychiatry 19...
14,,"Deer TR, Levy RM, Kramer J, Poree L, Amirdelfa...",Dorsal root ganglion stimulation yielded highe...,Pain,158(4):669-681,2017,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,http://www.ncbi.nlm.nih.gov/pubmed/28030470,,Multicenter (22 sites) 1:1 unblinded RCT,...,,NaN,NaN,NaN,JBS Oct 2017,NaN,NaN,pdf_downloads3/pdf_14.pdf,Research Paper\nDorsal root ganglion stimulati...,research paper dorsal root ganglion stimulatio...
15,,"De Carolis G, Paroli M, Tollapi L, Doust MW, B...",Paresthesia-independence: an assessment of tec...,Pain Physician,20(4):331-341,2017,https://www.painphysicianjournal.com/current/p...,https://www.ncbi.nlm.nih.gov/pubmed/28535555,,"Prospective, multi-center",...,,NaN,NaN,Rose Alzade for NEVRO 6/30/17,JBS 9/11/17,NaN,NaN,pdf_downloads3/pdf_15.pdf,Background: Spinal cord stimulation (SCS) has ...,background spinal cord stimulation sc successf...
19,,"Deer TR, Levy RM, Kramer J, Poree L, Amirdelfa...",Dorsal root ganglion stimulation yielded highe...,Pain,158(4):669-681,2017,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,http://www.ncbi.nlm.nih.gov/pubmed/28030470,,Multicenter (22 sites) 1:1 unblinded RCT,...,,NaN,NaN,JBS 25 Sept 2017,NaN,NaN,NaN,pdf_downloads3/pdf_19.pdf,Research Paper\nDorsal root ganglion stimulati...,research paper dorsal root ganglion stimulatio...
20,,"Slavin KV, North RB, D

In [36]:
%pip install keras

  Obtaining dependency information for keras from https://files.pythonhosted.org/packages/fe/58/34d4d8f1aa11120c2d36d7ad27d0526164b1a8ae45990a2fede31d0e59bf/keras-2.14.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.9 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [38]:
%pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/5a/f2/5c2f878c62c8b79c629b11b33516bb55054d7677eba6f56f3a20296b56bd/tensorflow-2.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.1.21 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.5 MB/s eta 0:00:00
  Obtaining dependency information for grpcio<2.0,>=1.24.3 from https://files.pythonhosted.org/packages/28/94/63bef715d0afea96662c66075d72249bf9b55b018b91b6937045bdc1470c/grpcio-1.58.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.

In [39]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.utils import to_categorical


# Split the dataset into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(
    data_with_full_text['full_article_text_cleaned'],
    data_with_full_text['Study design'],
    test_size=0.2,
    random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.2,
    random_state=42
)

# Preprocess the target 'Study design' text data
# You may need to encode the text labels into numerical values

# Tokenize and pad the input 'full_article_text_cleaned' data
max_words = 10000  # Maximum number of words to consider in the vocabulary
max_seq_length = 500  # Maximum sequence length (you can adjust this)

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

num_classes = len(data_with_full_text['Study design'].unique())

from sklearn.preprocessing import LabelEncoder

# Encode the target labels using LabelEncoder
label_encoder = LabelEncoder()

# Fit the label encoder on the entire 'Study design' column to ensure consistent encoding
label_encoder.fit(data_with_full_text['Study design'])

# Encode the labels for each split
y_train_encoded = label_encoder.transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)


y_train_encoded = to_categorical(y_train_encoded, num_classes=num_classes)
y_val_encoded = to_categorical(y_val_encoded, num_classes=num_classes)
y_test_encoded = to_categorical(y_test_encoded, num_classes=num_classes)


X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_seq_length)
X_val_pad = pad_sequences(X_val_seq, maxlen=max_seq_length)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_seq_length)

# Build a text classification model using LSTM
embedding_dim = 100  # Dimension of word embeddings (you can adjust this)

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_seq_length))
model.add(LSTM(64))
model.add(Dense(units=num_classes, activation='softmax'))  # num_classes is the number of unique 'Study design' labels

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
num_epochs = 10  # You can adjust the number of epochs
batch_size = 32  # You can adjust the batch size

model.fit(X_train_pad, y_train_encoded, validation_data=(X_val_pad, y_val_encoded), epochs=num_epochs, batch_size=batch_size)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_pad, y_test_encoded)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


2023-09-20 21:55:20.787426: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-20 21:55:20.836673: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-20 21:55:20.837975: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-20 21:55:21.938824: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-20 21:55:24.180246: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for ho

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 2.7764 - accuracy: 0.0000e+00 - val_loss: 2.7752 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 158ms/step - loss: 2.7620 - accuracy: 0.0909 - val_loss: 2.7787 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 159ms/step - loss: 2.7476 - accuracy: 0.8182 - val_loss: 2.7825 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 201ms/step - loss: 2.7328 - accuracy: 1.0000 - val_loss: 2.7865 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 159ms/step - loss: 2.7175 - accuracy: 1.0000 - val_loss: 2.7910 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 212ms/step - loss: 2.7015 - accuracy: 1.0000 - val_loss: 2.7958 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 179ms/step - loss: 2.6844 - accuracy: 1.0000 - val_loss: 2.8013 - val_accuracy: 0.

In [40]:
y_train

70     Crossover, 'double blind,' although suprathres...
5                             Prospective clinical study
11                                         Retrospective
104                                          Case report
53            Prospective, open-label, multicenter study
15                             Prospective, multi-center
35                                           Case series
106                                          Case report
19              Multicenter (22 sites) 1:1 unblinded RCT
102    Multicenter (10) 1:1 RCT (unblinded), stratifi...
21     Multicenter 1:1 RCT (note: previous publicatio...
Name: Study design, dtype: object